## Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import pandas as pd
import numpy as np
import re
import string
import spacy
import nltk
from nltk.corpus import stopwords as nltk_stopwords

from sklearn.model_selection import (train_test_split,GridSearchCV)
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import f1_score 
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import TfidfVectorizer 


from ipywidgets import IntProgress
from IPython.display import display
import time



In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
try:
    data=pd.read_csv(r"C:\Users\Admin\Desktop\toxic_comments.csv")
except:
    data=pd.read_csv('/datasets/toxic_comments.csv')

In [4]:
data.shape

(159292, 3)

In [5]:
data.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [7]:
data['text'].duplicated().sum()

0

In [8]:
data['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

In [9]:
string.punctuation=string.punctuation+'\n'

In [10]:
def remove_punctuation(text):
    no_punct=''.join([c for c in text if c not in string.punctuation])
    return no_punct

In [11]:
data['text'] = data['text'].apply(lambda x: remove_punctuation(x))

In [12]:
data.head()

,Unnamed: 0,text,toxic
0,0,ExplanationWhy the edits made under my usernam...,0
1,1,Daww He matches this background colour Im seem...,0
2,2,Hey man Im really not trying to edit war Its j...,0
3,3,MoreI cant make any real suggestions on improv...,0
4,4,You sir are my hero Any chance you remember wh...,0


In [13]:
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

In [14]:

prgBar = IntProgress(min = 0, max = data.shape[0]) # Создаем прогрессбар


In [15]:

def lemmatizer_text(text):
    doc = nlp(text)
    res=" ".join([token.lemma_ for token in doc])
    prgBar.value = prgBar.value + 1 
    return res

In [16]:
display(prgBar) # Выводим прогрессбар на экран

IntProgress(value=0, max=159292)

In [17]:
data['text'] = data['text'].apply(lambda x: lemmatizer_text(x))

In [18]:
features = data['text']
target = data['toxic']

In [19]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.2, random_state=12345)

In [20]:

corpus = features_train
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

count_tf_idf = TfidfVectorizer(stop_words=stopwords)
tf_idf = count_tf_idf.fit_transform(corpus) 

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [21]:

corpus_t = features_test
tf_idf_t = count_tf_idf.transform(corpus_t) 

## Обучение

## Логистическая регрессия

In [22]:

model=LogisticRegression(random_state = 12345,solver='liblinear')


parametrs = {'penalty': ['l1', 'l2']}
grid = GridSearchCV(model, parametrs, cv = 5, verbose = 0, n_jobs = -1, scoring='f1')

best_model = grid.fit(tf_idf, target_train)
LogisticRegression=best_model
print(grid.best_score_)



0.7543057500508175


## Случайный лес

In [23]:
model = RandomForestClassifier(random_state = 12345, verbose = False,class_weight ='balanced')
parametrs = {'max_depth' : range(1, 5, 1),
                'n_estimators' : range(1, 10, 1)
               
            }
grid = GridSearchCV(model, parametrs, cv=5, scoring='f1')
best_model=grid.fit(tf_idf, target_train)

print(grid.best_score_)

0.2745653721241522


## Решающее дерево

In [24]:
model = DecisionTreeClassifier(random_state = 12345)
parametrs = {'max_depth' : range(1, 5, 1)}
grid = GridSearchCV(model, parametrs, cv=5, scoring='f1')
best_model=grid.fit(tf_idf, target_train)

print(grid.best_score_)

0.46496347778132385


## Проверка модели

In [25]:
predicted_test=LogisticRegression.predict(tf_idf_t) 
f1 = f1_score(target_test,predicted_test)  
print(f1)

0.773561811505508


## Выводы

Наилучшая модель - логистическая регрессия, работает быстро и метрика f1>0.75. На тестовой выборке метрика равна 0.77.

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [X]  Весь код выполняется без ошибок
- [X]  Ячейки с кодом расположены в порядке исполнения
- [X]  Данные загружены и подготовлены
- [X]  Модели обучены
- [X]  Значение метрики *F1* не меньше 0.75
- [X]  Выводы написаны